# Using VAAS Outputs in Downstream Research

VAAS is designed as an inference-first library.
It does not train models or prescribe downstream tasks.

This notebook shows how researchers can:
- reuse VAAS anomaly scores and maps,
- integrate them into their own analysis pipelines,
- and treat VAAS as a modular signal rather than a full system.

All examples remain read-only with respect to VAAS internals.


## Installation

Install VAAS if it is not already available in the environment.


In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO
import numpy as np


## Loading the VAAS Pipeline

We load a pretrained VAAS model and use it as a fixed feature
and anomaly signal extractor.


In [ ]:
pipeline = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cpu",
    alpha=0.5,
)


## Loading an Example Image

The image is loaded from the VAAS GitHub repository to ensure
reproducibility across environments.


In [ ]:
image_url = (
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/"
    "examples/images/COCO_DF_C110B00000_00539519.jpg"
)

response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

image


## Extracting VAAS Outputs

VAAS returns both scalar scores and a spatial anomaly map.
These outputs can be reused independently in downstream tasks.


In [ ]:
output = pipeline(image)

s_f = output["S_F"]
s_p = output["S_P"]
s_h = output["S_H"]
anomaly_map = output["anomaly_map"]

print(f"S_F: {s_f:.4f}")
print(f"S_P: {s_p:.4f}")
print(f"S_H: {s_h:.4f}")
print("Anomaly map shape:", anomaly_map.shape)


## Using VAAS as a Scoring Signal

The scalar scores produced by VAAS can be treated as generic signals
in a larger system.

Examples include:
- ranking samples by visual inconsistency,
- filtering datasets before training,
- triggering human review or secondary analysis.

Because the scores are continuous, they integrate naturally
with statistical and decision-making pipelines.

---


## Using the Anomaly Map as a Spatial Signal

The anomaly map provides a spatial distribution of inconsistency.
While it is not a segmentation mask, it can still support analysis.

Below are simple examples of how researchers may summarize
or post-process the anomaly map.


---

In [ ]:
mean_anomaly = float(np.mean(anomaly_map))
max_anomaly = float(np.max(anomaly_map))
std_anomaly = float(np.std(anomaly_map))

print("Mean anomaly response:", mean_anomaly)
print("Max anomaly response:", max_anomaly)
print("Std anomaly response:", std_anomaly)


## Aggregating VAAS Outputs Across Images

In research settings, VAAS outputs are often aggregated across
many samples.

Common aggregation strategies include:
- mean or median S_H across a dataset,
- distribution analysis of anomaly scores,
- correlation with external labels or metadata.

These operations are intentionally left outside the VAAS API
to preserve flexibility.


Below is a minimal example of aggregating VAAS outputs across
multiple images.


In [ ]:
image_urls = [
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_C110B00000_00539519.jpg",
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_E000B00011_00834527.jpg",
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_E000B04010_00863031.jpg",
]

scores = []

for url in image_urls:
    img = Image.open(BytesIO(requests.get(url).content)).convert("RGB")
    out = pipeline(img)
    scores.append(out["S_H"])

scores = np.array(scores)

print("Dataset mean S_H:", scores.mean())
print("Dataset max  S_H:", scores.max())
print("Dataset std  S_H:", scores.std())


## What VAAS Deliberately Does Not Expose

The current VAAS API does not expose:
- intermediate feature tensors,
- raw attention weights,
- gradient information,
- or training-time hooks.

This separation ensures that VAAS remains:
- stable,
- reproducible,
- and suitable for forensic and audit-oriented workflows.

---

## Example Research Use Cases

Researchers have used VAAS-style outputs for:
- anomaly-aware dataset curation,
- explainability analysis in visual forensics,
- robustness evaluation under distribution shift,
- benchmarking anomaly sensitivity across models.

VAAS is best treated as a component rather than a full pipeline.

---


## Limitations

With the current VAAS API:
- VAAS outputs are inference-only,
- spatial maps are fixed-resolution,
- no training or fine-tuning is supported.

These constraints are intentional and documented.

---

## Summary

In this notebook, we:
- extracted VAAS scores and anomaly maps,
- demonstrated how they can be reused in downstream research,
- and clarified the scope and boundaries of the VAAS API.

The final notebook will summarize known limitations
and outline future research and API directions.

check it out: [10_known_limitations_and_future_research_directions.ipynb](https://colab.research.google.com/drive/1Vr2ufQp-pWwMh6tQt84DilYu6ESm-ZP2?usp=sharing)
